In [1]:
# If retraining is triggered:
# 1. Get the complete data from the database except the last 1 or 2 weeks data
# 2. Then preprocess and select features from this data and build the model
# 3. Compare the performance of the model (existing + new) on the last 1 or 2 weeks data to select the best model
# 4. Replace the deployed model file to the best model and other process files too (mapping dict and feature dict)

In [ ]:
%%capture

!pip install snowflake-connector-python

In [ ]:
%%capture

!pip install snowflake-sqlalchemy

In [ ]:
%%capture

!pip install xgboost

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import metrics

import sqlalchemy
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import *

import xgboost
from datetime import datetime, timedelta
import time  
import pytz    
tz_NY = pytz.timezone('Asia/Kolkata')

import snowflake_creds

import LOS_Preprocessing

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
        account="cr21746.ap-south-1",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="HEALTHSCHEMA"
    ))

In [3]:
def retraining_batch_query(a):
    query = f"""

        WITH TRAIN_BASE AS (

            SELECT CASE_ID,
                   COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
                   COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
                   COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
                   COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
                   COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
                   COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
                   COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
                   COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
                   COALESCE(BED_GRADE,0) AS BED_GRADE,
                   PATIENTID,
                   COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
                   COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
                   COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
                   COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
                   COALESCE(AGE,'None') AS AGE,
                   COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
                   ADMISSION_DATE,
                   DISCHARGE_DATE

            FROM HEALTHDB.HEALTHSCHEMA.HEALTH_DATA
            WHERE ADMISSION_DATE >= '2022-11-01' --- to reduce the load

        ),

        TRAIN_BASE_WITH_FEATURES AS (

            SELECT *,
                    MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
                    DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,    
                    CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
                    CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
                    CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
                    DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
            FROM TRAIN_BASE 

        ),    

        NEW_DATA_WITH_FEATURES AS (

             SELECT CASE_ID,
                       COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
                       COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
                       COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
                       COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
                       COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_X,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
                       COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
                       COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
                       COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
                       COALESCE(BED_GRADE,0) AS BED_GRADE,
                       PATIENTID,
                       COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
                       COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
                       COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
                       COALESCE(VISITORS_WITH_PATIENT_X,0) AS VISITORS_WITH_PATIENT,
                       COALESCE(AGE,'None') AS AGE,
                       COALESCE(ADMISSION_DEPOSIT_X,0) AS ADMISSION_DEPOSIT,
                       ADMISSION_DATE,
                       DISCHARGE_DATE,
                       ADMISSION_MONTH,
                       ADMISSION_DAY,
                       ADMISSION_ILLNESS,
                       ILLNESS_BEDGRADE,
                       DEPARTMENT_ILLNESS,
                       LOS_X AS LOS 
                    FROM HEALTHDB.HEALTHSCHEMA.TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
                    WHERE ADMISSION_DATE >= CURRENT_DATE-144+{a*7} AND ADMISSION_DATE < CURRENT_DATE-144+{(a+1)*7}    

        )


        SELECT * FROM TRAIN_BASE_WITH_FEATURES
        UNION ALL
        SELECT * FROM NEW_DATA_WITH_FEATURES;

        """
    return query

In [4]:
with engine.connect() as conn:
    data = pd.DataFrame(pd.read_sql(retraining_batch_query(0),conn))
    data.columns = [col.upper() for col in data.columns.tolist()]

In [5]:
print(data.shape)
data.head()

(63474, 25)


,CASE_ID,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,PATIENTID,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,VISITORS_WITH_PATIENT,AGE,ADMISSION_DEPOSIT,ADMISSION_DATE,DISCHARGE_DATE,ADMISSION_MONTH,ADMISSION_DAY,ADMISSION_ILLNESS,ILLNESS_BEDGRADE,DEPARTMENT_ILLNESS,LOS
0,5,26,b,2,Y,2,radiotherapy,S,D,2,31397,7,Trauma,Extreme,2,51-60,5558,2022-11-02,2022-12-22,Nov,Wed,Trauma-Extreme,Extreme-2,radiotherapy-Extreme,50
1,16,6,a,6,X,3,gynecology,Q,F,3,63418,8,Emergency,Extreme,2,71-80,9398,2022-11-21,2022-12-01,Nov,Mon,Emergency-Extreme,Extreme-3,gynecology-Extreme,10
2,23,21,c,3,Z,2,anesthesia,S,A,3,63418,8,Trauma,Extreme,2,71-80,6364,2022-11-03,2022-11-23,Nov,Thu,Trauma-Extreme,Extreme-3,anesthesia-Extreme,20
3,28,26,b,2,Y,4,gynecology,R,D,3,8088,2,Trauma,Moderate,2,31-40,5055,2022-11-08,2022-12-08,Nov,Tue,Trauma-Moderate,Moderate-3,gynecology-Moderate,30
4,42,27,a,7,Y,2,radiotherapy,S,C,3,84932,8,Emergency,Moderate,2,31-40,5038,2022-11-29,2022-12-09,Nov,Tue,Emergency-Moderate,Moderate-3,radiotherapy-Moderate,10


In [44]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

In [45]:
def feature_selection(df):
    # Creating X and Y
    x_train = df.drop('LOS',axis=1)
    y_train = df[['LOS']]
    
    # Decision Tree
    from sklearn.tree import DecisionTreeRegressor

    dtree = DecisionTreeRegressor()
    dtree.fit(x_train,y_train)
    # Feature Importance
    feat_imp = (pd.DataFrame(zip(x_train.columns,dtree.feature_importances_),columns=['feature','imp'])
                .sort_values(by='imp',ascending=False))
    final_features_dtree = feat_imp[feat_imp['imp']>=0.01]['feature'].values.tolist()
    
    # XGBoost
    import xgboost as xgb

    xgb_ = xgb.XGBRegressor()
    xgb_.fit(x_train,y_train)
    # Feature Importance
    feat_imp = (pd.DataFrame(zip(x_train.columns,xgb_.feature_importances_),columns=['feature','imp'])
                .sort_values(by='imp',ascending=False))
    final_features_xgb = feat_imp[feat_imp['imp']>=0.01]['feature'].values.tolist()
    
    model_features =  list(set(final_features_dtree).union(set(final_features_xgb)))
    print("Final Features from both Dtree & XGB: "+str(len(model_features)))
    
    import pickle

    final_feats_list = model_features+['LOS']

    with open('./Retraining Artifacts/MODEL_FEATS.pkl','wb') as F:
        pickle.dump(final_feats_list,F)
    
    return final_feats_list
    

In [48]:
# model_feats.remove('LOS')

['ADMISSION_DAY_Thu',
 'WARD_TYPE_S',
 'ADMISSION_DEPOSIT',
 'AGE_41-50',
 'BED_GRADE_2',
 'WARD_TYPE_R',
 'DEPARTMENT_ILLNESS_gynecology-Extreme',
 'ADMISSION_DAY_Wed',
 'ADMISSION_DAY_Fri',
 'ADMISSION_DAY_Tue',
 'ADMISSION_DAY_Sun',
 'AGE_21-30',
 'ILLNESS_BEDGRADE_Moderate-2',
 'WARD_TYPE_P',
 'AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL',
 'TYPE_OF_ADMISSION_Emergency',
 'ADMISSION_DAY_Sat',
 'WARD_TYPE_Q',
 'CITY_CODE_HOSPITAL_7',
 'CITY_CODE_HOSPITAL_13',
 'AGE_51-60',
 'AGE_31-40',
 'AGE_61-70',
 'TYPE_OF_ADMISSION_Trauma',
 'VISITORS_WITH_PATIENT',
 'CITY_CODE_PATIENT_2',
 'CITY_CODE_PATIENT_8',
 'ADMISSION_DAY_Mon',
 'AGE_71-80',
 'CITY_CODE_PATIENT_21',
 'CITY_CODE_HOSPITAL_2',
 'ILLNESS_BEDGRADE_Extreme-1',
 'SEVERITY_OF_ILLNESS_Minor']

In [49]:
with engine.connect() as conn:
    
    # Loading the scoring data
    data = pd.DataFrame(pd.read_sql(retraining_batch_query(0),conn))
    data.columns = [col.upper() for col in data.columns.tolist()]
    print(data.shape)
#     display(data.head())
    
    # Splitting the data into Train and Test set
    import pytz    
    from datetime import datetime, timedelta
    tz_NY = pytz.timezone('Asia/Kolkata')

    max_date = data.ADMISSION_DATE.max()
    min_date = max_date - timedelta(days=7)
    
    data_train = data[(data['ADMISSION_DATE'] <= min_date)]
    data_test = data[(data['ADMISSION_DATE'] >= min_date) & (data['ADMISSION_DATE'] <= max_date)]
    
    
    # Applying the preprocessing steps
    df_train_processed = LOS_Preprocessing.preprocess_data(data_train)
    print(df_train_processed.shape)
    
    df_test_processed = LOS_Preprocessing.preprocess_data(data_test)
    print(df_test_processed.shape)
    
    # Performing feature selection
    df_final = df_train_processed.copy()
    print(df_final.shape)
#     display(df_final.head())
    print("Feature Selection Started..")
    model_feats = feature_selection(df_final)
    print(model_feats)
    model_feats.remove('LOS')
    
    # Model Building
    import xgboost as xgb

    xgb_ = xgb.XGBRegressor()
    xgb_.fit(df_final[model_feats],df_final['LOS'])
    
    df_test_final = check_n_create_model_features(df_test_processed,model_feats)
    if 'LOS' in df_test_final.columns.tolist():
        df_test_final = df_test_final.drop('LOS',axis=1)
    preds = np.ceil(xgb_.predict(df_test_final))
    rmse = np.sqrt(metrics.mean_squared_error(df_test_processed['LOS'],preds))
    mae = np.sqrt(metrics.mean_absolute_error(df_test_processed['LOS'],preds))
    print("\n Test Performance (new model)")
    print("RMSE: ", rmse)
    print("MAE: ", mae)      
    
    # Saving the trained model
    booster = xgb_.get_booster()
    booster.save_model('./Retraining Artifacts/MODEL_XGB.model')
    
    model_xgb_metrics_new = {}
    model_xgb_metrics_new['RMSE'] = rmse
    model_xgb_metrics_new['MAE'] = mae

    import pickle

    with open('./Retraining Artifacts/MODEL_XGB_PERFM_METRICS.pkl','wb') as F:
        pickle.dump(model_xgb_metrics_new,F)
    
    
    # Getting the predictions from the old model
    model = xgboost.XGBRegressor()
    model.load_model('MODEL_XGB.model')
#     df_test_processed['PREDICTED_LOS'] = np.ceil(model.predict(df_test_processed[model_feats]))

    with open('MODEL_FEATS.pkl','rb') as F:
        model_feats_old = pickle.load(F)
        
    df_test_final = check_n_create_model_features(df_test_processed,model_feats_old)
    if 'LOS' in df_test_final.columns.tolist():
        df_test_final = df_test_final.drop('LOS',axis=1)
    preds = np.ceil(model.predict(df_test_final))
    rmse = np.sqrt(metrics.mean_squared_error(df_test_processed['LOS'],preds))
    mae = np.sqrt(metrics.mean_absolute_error(df_test_processed['LOS'],preds))
    print("\n Test Performance (old model)")
    print("RMSE: ", rmse)
    print("MAE: ", mae)   
    
    model_xgb_metrics_old = {}
    model_xgb_metrics_old['RMSE'] = rmse
    model_xgb_metrics_old['MAE'] = mae
    

(63474, 25)
(46737, 146)
(16737, 140)
(46737, 146)
Feature Selection Started..
Final Features from both Dtree & XGB: 33
['ADMISSION_DAY_Thu', 'WARD_TYPE_S', 'ADMISSION_DEPOSIT', 'AGE_41-50', 'BED_GRADE_2', 'WARD_TYPE_R', 'DEPARTMENT_ILLNESS_gynecology-Extreme', 'ADMISSION_DAY_Wed', 'ADMISSION_DAY_Fri', 'ADMISSION_DAY_Tue', 'ADMISSION_DAY_Sun', 'AGE_21-30', 'ILLNESS_BEDGRADE_Moderate-2', 'WARD_TYPE_P', 'AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL', 'TYPE_OF_ADMISSION_Emergency', 'ADMISSION_DAY_Sat', 'WARD_TYPE_Q', 'CITY_CODE_HOSPITAL_7', 'CITY_CODE_HOSPITAL_13', 'AGE_51-60', 'AGE_31-40', 'AGE_61-70', 'TYPE_OF_ADMISSION_Trauma', 'VISITORS_WITH_PATIENT', 'CITY_CODE_PATIENT_2', 'CITY_CODE_PATIENT_8', 'ADMISSION_DAY_Mon', 'AGE_71-80', 'CITY_CODE_PATIENT_21', 'CITY_CODE_HOSPITAL_2', 'ILLNESS_BEDGRADE_Extreme-1', 'SEVERITY_OF_ILLNESS_Minor', 'LOS']

 Test Performance (new model)
RMSE:  18.646845458462177
MAE:  3.547406816055851

 Test Performance (old model)
RMSE:  14.908764378801582
MAE:  3.3061688138

In [50]:
# df_test_final

In [16]:
model.n_features_in_

27

In [17]:
len(model_feats_old)

28

# Retraining Final Scripts:

In [52]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

In [77]:
def feature_selection(df):
    # Creating X and Y
    x_train = df.drop('LOS',axis=1)
    y_train = df[['LOS']]
    
    # Decision Tree
    from sklearn.tree import DecisionTreeRegressor

    dtree = DecisionTreeRegressor()
    dtree.fit(x_train,y_train)
    # Feature Importance
    feat_imp = (pd.DataFrame(zip(x_train.columns,dtree.feature_importances_),columns=['feature','imp'])
                .sort_values(by='imp',ascending=False))
    final_features_dtree = feat_imp[feat_imp['imp']>=0.01]['feature'].values.tolist()
    
    # XGBoost
    import xgboost as xgb

    xgb_ = xgb.XGBRegressor()
    xgb_.fit(x_train,y_train)
    # Feature Importance
    feat_imp = (pd.DataFrame(zip(x_train.columns,xgb_.feature_importances_),columns=['feature','imp'])
                .sort_values(by='imp',ascending=False))
    final_features_xgb = feat_imp[feat_imp['imp']>=0.01]['feature'].values.tolist()
    
    model_features =  list(set(final_features_dtree).union(set(final_features_xgb)))
    print("Final Features from both Dtree & XGB: "+str(len(model_features)))
    
    import pickle

    final_feats_list = model_features+['LOS']

    with open('./Retraining Artifacts/MODEL_FEATS.pkl','wb') as F:
        pickle.dump(final_feats_list,F)
    
    return final_feats_list
    

In [78]:
def retraining_batch_query(max_date):
    query = f"""

        WITH TRAIN_BASE AS (

            SELECT CASE_ID,
                   COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
                   COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
                   COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
                   COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
                   COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
                   COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
                   COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
                   COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
                   COALESCE(BED_GRADE,0) AS BED_GRADE,
                   PATIENTID,
                   COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
                   COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
                   COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
                   COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
                   COALESCE(AGE,'None') AS AGE,
                   COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
                   ADMISSION_DATE,
                   DISCHARGE_DATE

            FROM HEALTHDB.HEALTHSCHEMA.HEALTH_DATA
            WHERE ADMISSION_DATE >= '2022-11-01' --- To reduce the load

        ),

        TRAIN_BASE_WITH_FEATURES AS (

            SELECT *,
                    MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
                    DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,    
                    CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
                    CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
                    CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
                    DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
            FROM TRAIN_BASE 

        ),    

        NEW_DATA_WITH_FEATURES AS (

             SELECT CASE_ID,
                       COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
                       COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
                       COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
                       COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
                       COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_X,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
                       COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
                       COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
                       COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
                       COALESCE(BED_GRADE,0) AS BED_GRADE,
                       PATIENTID,
                       COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
                       COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
                       COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
                       COALESCE(VISITORS_WITH_PATIENT_X,0) AS VISITORS_WITH_PATIENT,
                       COALESCE(AGE,'None') AS AGE,
                       COALESCE(ADMISSION_DEPOSIT_X,0) AS ADMISSION_DEPOSIT,
                       ADMISSION_DATE,
                       DISCHARGE_DATE,
                       ADMISSION_MONTH,
                       ADMISSION_DAY,
                       ADMISSION_ILLNESS,
                       ILLNESS_BEDGRADE,
                       DEPARTMENT_ILLNESS,
                       LOS_X AS LOS 
                    FROM HEALTHDB.HEALTHSCHEMA.TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
                    WHERE ADMISSION_DATE < '{max_date}'

        )


        SELECT * FROM TRAIN_BASE_WITH_FEATURES
        UNION ALL
        SELECT * FROM NEW_DATA_WITH_FEATURES;

        """
    return query

In [79]:
def retrain_model(cut_off_date):
    with engine.connect() as conn:
    
        # Loading the scoring data
        data = pd.DataFrame(pd.read_sql(retraining_batch_query(cut_off_date),conn))
        data.columns = [col.upper() for col in data.columns.tolist()]
        print(data.shape)
    #     display(data.head())

        # Splitting the data into Train and Test set
        import pytz    
        from datetime import datetime, timedelta
        tz_NY = pytz.timezone('Asia/Kolkata')

        max_date = data.ADMISSION_DATE.max()
        min_date = max_date - timedelta(days=7)

        data_train = data[(data['ADMISSION_DATE'] <= min_date)]
        data_test = data[(data['ADMISSION_DATE'] >= min_date) & (data['ADMISSION_DATE'] <= max_date)]


        # Applying the preprocessing steps
        df_train_processed = LOS_Preprocessing.preprocess_data(data_train)
        print(df_train_processed.shape)

        df_test_processed = LOS_Preprocessing.preprocess_data(data_test)
        print(df_test_processed.shape)

        # Performing feature selection
        df_final = df_train_processed.copy()
        print(df_final.shape)
    #     display(df_final.head())
        print("Feature Selection Started..")
        model_feats = feature_selection(df_final)
        print(model_feats)
        model_feats.remove('LOS')

        # Model Building
        import xgboost as xgb

        xgb_ = xgb.XGBRegressor()
        xgb_.fit(df_final[model_feats],df_final['LOS'])

        df_test_final = check_n_create_model_features(df_test_processed,model_feats)
        if 'LOS' in df_test_final.columns.tolist():
            df_test_final = df_test_final.drop('LOS',axis=1)
        preds = np.ceil(xgb_.predict(df_test_final))
        rmse = np.sqrt(metrics.mean_squared_error(df_test_processed['LOS'],preds))
        mae = np.sqrt(metrics.mean_absolute_error(df_test_processed['LOS'],preds))
        print("\n Test Performance (new model)")
        print("RMSE: ", rmse)
        print("MAE: ", mae)      

        # Saving the trained model
        booster = xgb_.get_booster()
        booster.save_model('./Retraining Artifacts/MODEL_XGB.model')

        model_xgb_metrics_new = {}
        model_xgb_metrics_new['RMSE'] = rmse
        model_xgb_metrics_new['MAE'] = mae

        import pickle

        with open('./Retraining Artifacts/MODEL_XGB_PERFM_METRICS.pkl','wb') as F:
            pickle.dump(model_xgb_metrics_new,F)


        # Getting the predictions from the old model
        model = xgboost.XGBRegressor()
        model.load_model('MODEL_XGB.model')
    #     df_test_processed['PREDICTED_LOS'] = np.ceil(model.predict(df_test_processed[model_feats]))

        with open('MODEL_FEATS.pkl','rb') as F:
            model_feats_old = pickle.load(F)

        df_test_final = check_n_create_model_features(df_test_processed,model_feats_old)
        if 'LOS' in df_test_final.columns.tolist():
            df_test_final = df_test_final.drop('LOS',axis=1)
        preds = np.ceil(model.predict(df_test_final))
        rmse = np.sqrt(metrics.mean_squared_error(df_test_processed['LOS'],preds))
        mae = np.sqrt(metrics.mean_absolute_error(df_test_processed['LOS'],preds))
        print("\n Test Performance (old model)")
        print("RMSE: ", rmse)
        print("MAE: ", mae)   

        model_xgb_metrics_old = {}
        model_xgb_metrics_old['RMSE'] = rmse
        model_xgb_metrics_old['MAE'] = mae
    
    return model_xgb_metrics_new, model_xgb_metrics_old


In [80]:
new_dict, old_dict = retrain_model('2022-12-30')

(116062, 25)
(99271, 147)
(19218, 144)
(99271, 147)
Feature Selection Started..
Final Features from both Dtree & XGB: 25
['WARD_TYPE_P', 'ADMISSION_DAY_Thu', 'AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL', 'TYPE_OF_ADMISSION_Emergency', 'WARD_TYPE_Q', 'CITY_CODE_HOSPITAL_7', 'WARD_TYPE_S', 'ADMISSION_DEPOSIT', 'AGE_41-50', 'BED_GRADE_2', 'AGE_31-40', 'ADMISSION_MONTH_Dec', 'TYPE_OF_ADMISSION_Trauma', 'VISITORS_WITH_PATIENT', 'ADMISSION_MONTH_Nov', 'CITY_CODE_PATIENT_8', 'DEPARTMENT_ILLNESS_gynecology-Extreme', 'ADMISSION_DAY_Wed', 'AGE_71-80', 'ADMISSION_DAY_Fri', 'ADMISSION_DAY_Sun', 'ADMISSION_DAY_Tue', 'CITY_CODE_HOSPITAL_2', 'ILLNESS_BEDGRADE_Extreme-1', 'SEVERITY_OF_ILLNESS_Minor', 'LOS']

 Test Performance (new model)
RMSE:  15.035353972589732
MAE:  3.3048134697641314

 Test Performance (old model)
RMSE:  14.951689643896763
MAE:  3.297176153934122


In [81]:
def finalize_model(new_perform_dict, old_perform_dict):
    count = 0
    for metric in new_perform_dict.keys():
        if new_perform_dict[metric] < old_perform_dict[metric]:
            count += 1
    
    if count > 0:
        return 'New Model'
    else:
        return 'Old Model'
        

In [82]:
finalize_model(new_perform_dict=new_dict, old_perform_dict=old_dict)

'Old Model'

In [83]:
def deploy_model(selector='Old Model'):
    if selector != 'Old Model':
        # STEP-1:
        # Loading the old model
        with open('MODEL_FEATS.pkl','rb') as F:
            old_feats = pickle.load(F)
        with open('MODEL_XGB.model','rb') as F:
            old_model = pickle.load(F)
        with open('MODEL_XGB_PERFM_METRICS.pkl','rb') as F:
            old_perfm_dict = pickle.load(F)
        
        # Saving the copy to Archive folder
        with open('./Archive/MODEL_FEATS.pkl','wb') as F:
            pickle.dump(old_feats,F)
        with open('./Archive/MODEL_XGB.model','wb') as F:
            pickle.dump(old_model,F)
        with open('./Archive/MODEL_XGB_PERFM_METRICS.pkl','wb') as F:
            pickle.dump(old_perfm_dict,F)
        
        # STEP-2:
        # Loadin the new model
        with open('./Retraining Artifacts/MODEL_FEATS.pkl','rb') as F:
            new_feats = pickle.load(F)
        with open('./Retraining Artifacts/MODEL_XGB.model','rb') as F:
            new_model = pickle.load(F)
        with open('./Retraining Artifacts/MODEL_XGB_PERFM_METRICS.pkl','rb') as F:
            new_perfm_dict = pickle.load(F)
        
        # Replacing the old model artifacts with the new model
        with open('MODEL_FEATS.pkl','wb') as F:
            pickle.dump(new_feats,F)
        with open('MODEL_XGB.model','wb') as F:
            pickle.dump(new_model,F)
        with open('MODEL_XGB_PERFM_METRICS.pkl','wb') as F:
            pickle.dump(new_perfm_dict,F)
        
    return 'Deployment Successful'

In [84]:
deploy_model('Old Model')

'Deployment Successful'